In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from fastai.vision import *
from fastai.metrics import accuracy
from fastai.basic_data import *
from skimage.util import montage
import pandas as pd
from torch import optim
import re
from pathlib import Path
from functools import partial
import multiprocessing

from utils import *

With simpler model, my CPU cannot keep up with feeding the GPU due to how long resizing takes.

Let's resize the images to speed things up.

In [2]:
SZ = 224
n_process = 6

In [3]:
train_id = 'train1'
test_id = 'test1'

PATH = Path('../input')

src_train_dir = PATH/train_id
src_test_dir = PATH/test_id

dst_train_dir = PATH/f'{train_id}_{SZ}'
dst_test_dir = PATH/f'{test_id}_{SZ}'

In [4]:
from concurrent.futures import ThreadPoolExecutor

In [5]:
def resize_img(src, dst_dir):
    dst_file = dst_dir/os.path.basename(src)
    PIL.Image.open(src).resize((SZ,SZ), resample=PIL.Image.BICUBIC).save(dst_file)

In [6]:
def resize_img_dir(src_dir, dst_dir, n_process=4):
    dst_dir.mkdir(exist_ok=True)
    src_files = list((src_dir).iterdir())
    resize_img_to_dst_dir = partial(resize_img, dst_dir=dst_dir)
    with multiprocessing.Pool(n_process) as pool:
        pool.map(resize_img_to_dst_dir, src_files)

In [7]:
%%time
print('Resizing train files ...')
resize_img_dir(src_train_dir, dst_train_dir, n_process)
print('Resizing test files ...')
resize_img_dir(src_test_dir, dst_test_dir, n_process)

Resizing train files ...
Resizing test files ...
CPU times: user 20 ms, sys: 52 ms, total: 72 ms
Wall time: 482 ms


In [8]:
len(list((dst_train_dir).iterdir())), len(list((dst_test_dir).iterdir()))

(58, 6)